# Gestion des graphiques avec R

R offers two main graphical systems: `base`  and `grid`. The latter is exposed in two core packages: `lattice` and `ggplot2`. We will use the later, which relies on the idea of a "Grammar of Graphics".

In [ ]:
suppressPackageStartupMessages(library(ggplot2))
theme_set(theme_bw())

The above instructions allow to load the required package and to set a default theme. They are meant to be run only once, when R is started for the current session. However, it is still possible to change the theme at any time, or inline when building a custom graphical display.

Let's look at a random sample of the GSOEP dataset available in `data/gsoep09.dta`. This is a Stata file built upon the [German Socio Economic Survey](https://www.eui.eu/Research/Library/ResearchGuides/Economics/Statistics/DataPortal/GSOEP) from 2009. Note that these data come with survey weights (`dweight` and `xweights`) but we will proceed as if it was a cross-sectional sample. The `foreign` package allows to read Stata files (up to Stata 12 version), but it is more convenient to use the `haven` package. The `read_dta` function will return a "tibble", which is like an ordinary data frame but with extra properties that we won't exploit in this tutorial.

In [ ]:
library(haven)
d <- read_dta("data/gsoep09.dta")
head(d)

## Data preprocessing

We will first subset the data frame by selecting only a dozen of variables, and then draw a random sample of 10% of the original dataset. Specifically, the variables we are interested in are described below:

- `persnr`: respondant ID
- `hhnr2009`: household ID
- `ybirth`: year of birth
- `sex`: sex of respondant
- `mar`: marital status
- `egp`: socio-economic class
- `yedu`: no. years of education
- `income`: annual income (€)
- `rel2head`: position of respondant relative to household 
- `wor01` to `wor12`: 3-point Likert answers to socio-economic and political questions

In [ ]:
vars <- c("persnr", "hhnr2009", "ybirth", "sex", "mar", "egp", "yedu", "income", "rel2head", 
          "wor01", "wor02", "wor03", "wor04", "wor05", "wor06", "wor07", "wor08", "wor09", "wor10", "wor11", "wor12")
set.seed(101)
idx <- sample(1:nrow(d), floor(nrow(d)*.1))
d <- subset(d[idx, ], select = vars)
dim(d)

The next step consists in re-encoding categorical variables and computing auxiliary variables:

In [ ]:
d$persnr <- factor(d$persnr)
d$hhnr2009 <- factor(d$hhnr2009)
d$sex <- droplevels(as_factor(d$sex))
d$mar <- droplevels(as_factor(d$mar))
d$egp <- droplevels(as_factor(d$egp))
d$rel2head <- droplevels(as_factor(d$rel2head))
d$age <- 2009 - d$ybirth

Let us now look at the above variables, and recode some of `mar` and `egp` categories: (For simplicity, we will discard all refusals from the present dataset.)

In [ ]:
table(d$mar)
levels(d$mar)[3:5] <- "Single"
d$mar[d$mar == "Refusal"] <- NA
d$mar <- droplevels(d$mar)
table(d$mar)

In [ ]:
table(d$egp)
levels(d$egp)[1:2] <- "High"
levels(d$egp)[2:4] <- "Mid"
levels(d$egp)[3:4] <- "Low"
levels(d$egp)[4:6] <- "None"
d$egp[d$egp == "Refusal"] <- NA
d$egp <- droplevels(d$egp)
table(d$egp)

Finally, let us only keep individuals with available income, and no missing value on `mar` or `egp`:

In [ ]:
d <- subset(d, income > 0 & !is.na(mar) & !is.na(egp))
d$logincome <- log(d$income)
dim(d)

## The GGplot2 philosophy

In the spirit of the *Grammar of Graphics* developped by Leland Wilkinson, the `ggplot2` library uses a system of layers where graphical elements are joined altogether in a coherent way.

![](assets/lang-r-ggplot-001.png)

The following elements are usually found, in more or less the following order:

- `ggplot()`: a data frame (`data=`) together with a mapping (`aes()`)
- `geom_*()`: one or more geometrical objects
- `facet_wrap()`: a system of facets (used for conditioning on additional variables)
- `scale_*_*()`: a custom scale for each axis and color schemes
- `coord_*()`: a coordinate system
- `labs()`: some annotations for axes and other graphical properties
- `theme_*()`: a custom theme

Basically, we start by indicating the data frame in which the variables can be found (`data =`), and what role these variables play in the plot (`aes(x=, y=, color=)`). Two numerical variables can be used to code the spatial location of a point in a 2D space, while a categorical variable could be used to highligtht those points using color values mapped onto variable levels. The type of graphical object we wan to draw (`geom_point()`, `geom_histogram()`) depends on the number and the type of variables available in the `aes()` mapping. If there's only one numeric variable, we cannot draw a scatterplot, of course, but we can build an histogram, for example. Note that a single plot can contain multiple geometrical object, e.g. a scatterplot and a scatterplot smoother (lowess curve). Each plot can be customized in several ways, but most of the time we may be interested in updating the axes or the object properties (e.g., `scale_x_continuous()`, `scale_color_manual()`), and the labels or the title/subtitle (`labs()`).

The most up to date documentation is available on-line: Winston Chang, *[R Graphics Cookbook](https://r-graphics.org)* (2nd ed.).

### Illustration of the layered approach

A very basic plot can be thought of as a succession of layers. In the example below, we first draw a scatterplot using variable `lwt` (aon the x-axis) and `bwt` (on the y-axis) from the `MASS::birthwt` dataset, and then add a scatterplot smoother (a lowess curve, with default smoothing parameters) on top of the scatterplot:

In [ ]:
p <- ggplot() +
    layer(data = MASS::birthwt,
          stat = "identity",
          geom = "point",
          mapping = aes(x = lwt, y = bwt),
          position = "identity") +
    layer(data = MASS::birthwt,
          stat = "smooth",
          geom = "line",
          mapping = aes(x = lwt, y = bwt),
          position = "identity",
          params = list(method = "auto"))

Here is the simplified version using the approach described above:

In [ ]:
library(MASS)
p <- ggplot(data = birthwt, aes(x = lwt, y = bwt))
p + geom_point() + geom_smooth(method = "auto")

This follows the same principled approach: You add each graphical piece together using the `+` operateur -- which is specific to `ggplot2` -- and you print the final graphical object to the graphical device. This is the reason why we don't use any assignment operateur (`<-`) in the last expression. It would be possible to write the following expression:

    ggplot(data = birthwt, aes(x = lwt, y = bwt)) + geom_point() + geom_smooth(method = "auto")
    
However, it is better practice to save the graphical commands in a variable, and even to accumulate (`+`) the instructions as they go along since this allows to build the final plot in an incremental way.

## Exploratory analysis

### Histogram and density estimators

Histogram and density curve are two common estimators for the distribution of a continuous random variable. They do indeed have their counterparts in `ggplot2`, namely `geom_histogram()` and `geom_density()`, although in the latter case it is also possible to use `geom_line` using the a density estimator that `ggplot2` will copute for us. Here are two examples of use:

In [ ]:
p <- ggplot(data = d, aes(x = age)) +
  geom_histogram(binwidth = 5) +
  labs(x = "Age", y = "Counts")
p

In [ ]:
p <- ggplot(data = d, aes(x = age)) +
  geom_line(stat = "density") +
  labs(x = "Age", y = "Density")
p

The syntax is almost identical in both case, pending the options specific to each geometrical object: In the case of `geom_histogram`, it is possible to indicate the size (`binwidth=`) or the number (`bins=`) of intervals to use when discretizing the continuous variable `x`; with the density estimator, we can specify the size of the smoothing window (`bw=` or `adjust=`) and/or the kernel (`kernel=`).

### Boxplots

Boxplots can be used to visually depict the output of the `summary` function, that is a five-number summary of the distribution of a numerical variable (location, shape and range). It is most useful when there are several distributions to plot side by side, hence the use of `x=` and `y=` in the mapping below:

In [ ]:
p <- ggplot(data = d, aes(x = mar, y = logincome)) +
  geom_boxplot() +
  labs(x = "Marital Status", y = "Annual Income (log)") +
  coord_flip()
p

### Barplots

Let us summarize the distribution of average (log) income across socio-economic classes. First, we need to compute the mean and standard deviation of `logincome` for each level of `egp`. This is easily performed using `aggregate` (or `tapply`):

In [ ]:
egp_stats <- aggregate(logincome ~ egp, data = d, mean)
egp_stats$sd <- aggregate(logincome ~ egp, data = d, sd)$logincome
names(egp_stats)[2] <- "mean"
egp_stats

Unfortunately, `aggregate` returns a data frame with only one column for the computed results (named after the variable that appears in the LHS), no matter if the function returns a single valued vector or multivariate vectors. Hence, we are forced to extract the result computed with the second call to `aggregate` and append it as a distinct column. It is then possible to use a combination of `geom_bar` and `geom_errorbar` to produce the desired result:

In [ ]:
p <- ggplot(data = egp_stats, aes(x = egp, y = mean)) +
  geom_bar(stat = "identity") +
  geom_errorbar(aes(ymin = mean - sd, ymax = mean + sd), width = .2, col = "black") +
  labs(x = "Socio-Economic Status", y = "Average income (log)")
p

Here is another example using two variables in the RHS, `egp` and `sex`:

In [ ]:
egp_stats <- aggregate(logincome ~ egp + sex, data = d, mean)
egp_stats$sd <- aggregate(logincome ~ egp + sex, data = d, sd)$logincome
names(egp_stats)[3] <- "mean"

In [ ]:
p <- ggplot(data = egp_stats, aes(x = egp, y = mean, fill = sex)) +
  geom_bar(stat = "identity", position = position_dodge()) +
  geom_errorbar(aes(ymin = mean - sd, ymax = mean + sd), width = .2, col = "black", position = position_dodge(.9)) +
  scale_fill_manual("", values = c("cornflowerblue", "darkorange")) +
  labs(x = "Socio-Economic Status", y = "Average income (log)")
p

### Scatterplots

Scatterplot are a common way to display the joint variation of two series of observations, specified as `x=` and `y=` coordinates in a 2D plan. A scatterplot smoother, e.g. a lowess (or loess) curve, can be superimposed on the scatterplot to help in gauging local deviation to the linear relationship assumption commonly found in linear regression. In addition to the `x` and `y` variables, it is also possible to ask `ggplot2` to highlight point using a color that depends on the level of a third categorical variable. This is controlled in the `aes`thetic directly. When we are not happy with the default color scheme, we can provide our own values (RGB, HEX or named colors) using a "manual" scale, which targets an existing aesthetic. Finally, `facet_grid()` or `facet_warp()` can be used to build a treillis graphic, meaning that the overall plot is subsetted according to the facet defined in this specific instruction. Here is a full working example in action:

In [ ]:
p <- ggplot(data = subset(d, egp != "None"), aes(x = age, y = logincome, color = sex)) +
  geom_point(alpha = 0.5) +
  geom_smooth(method = "loess", se = FALSE, span = 1.5) +
  scale_color_manual("", values = c("cornflowerblue", "darkorange")) +
  facet_grid(mar ~ egp) +
  labs(x = "Age", y = "Average income (log)")
p